In [1]:
import re, os, time
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
from gensim.models.ldamodel import LdaModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Import nltk stopwords and spacy for lemmatization
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer

import spacy
# Enable logging for Gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings('ignore',category=DeprecationWarning)

import time
import pickle
import bs4

# Collect names of Tweet files
from os import listdir
from os.path import isfile, join
import json

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
import textacy
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from bokeh.charts import Chord

from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
from IPython.display import display

from pysankey import sankey
from IPython.display import (
    Image,
    SVG
)

%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
The bokeh.charts API has moved to a separate 'bkcharts' package.

This compatibility shim will remain until Bokeh 1.0 is released.
After that, if you want to use this API you will have to install
the bkcharts package explicitly.

  warn(message)


## Data Pre-processing part (uncomment this section to do preprocessing part)

In [2]:
# mypath = 'data/election'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [3]:
# # Collect all Tweets available and store them as list of dicts

# all_tweets = []

# for f in onlyfiles:
#     full_name = mypath+'/'+f

#     with open(full_name, 'r') as f:
#         data = json.load(f)
#         all_tweets.extend(data)

In [4]:
# all_tweets_df = pd.DataFrame(all_tweets)
## Select only dutch tweets
# all_tweets_df = all_tweets_df[all_tweets_df['lang'] == 'nl']

In [5]:
# print('All tweets :',len(all_tweets))

In [6]:
# all_tweets = all_tweets_df.to_dict('records')
# print('All dutch only tweets :',len(all_tweets))

In [7]:
# # Save collected Tweets
# with open('elections_tweets', 'w') as fout:
#     json.dump(all_tweets, fout)

#### Pre-Loaded and pre-pre-processed collected Tweets

In [8]:
# # Load collected Tweets
# import json
# with open('elections_tweets', 'r') as fout:
#         all_tweets = json.load(fout)

In [9]:
# # Statistics
# print('Sample of a Tweet:', all_tweets[0]['text'], '\n')
# print('Number of fields available:', len(all_tweets[0].keys()), '\n')
# print('Fields available in a Tweet:', '\n', [*all_tweets[0]], '\n')
# print('Info avaialable in the whole set:', '\n', pd.DataFrame(all_tweets[:10]).columns.values)

Separating tweet to list and dict

In [10]:
# # Extract the data (id and text), and put it into dict and list
# text_dict = {} 
# text_list = []
# id_list = []
# missing_tweets = 0

# for tweet in all_tweets:
#     if ('text' in tweet.keys()):
#         if ('id' in tweet.keys()):
#             my_id = tweet['id']
#             if (tweet['text'] != None):
#                 text_dict[my_id] = tweet['text']
#                 id_list.append(my_id)
#                 text_list.append([tweet['text']])
                
#             elif (tweet['text'] == None):
#                 missing_values = missing_tweets + 1

In [11]:
# # Save processed Tweets
# with open('elections_text_list', 'wb') as fp:
#     pickle.dump(text_list, fp)
    
# with open('elections_text_dict', 'wb') as fp:
#     pickle.dump(text_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

#### Pre-Loaded and pre-pre-processed text list and dict

In [12]:
# # Load processed Tweets
# with open('elections_text_list', 'rb') as fp:
#     text_list = pickle.load(fp)
    
# with open('elections_text_dict', 'rb') as fp:
#     text_dict = pickle.load(fp)

In [13]:
# # Text Statistics
# t = len(text_dict)
# max_length = 0
# average_length = 0

# for tweet in text_list : 
#     average_length = average_length + sum(len(i) for i in tweet) 
#     if (sum(len(i) for i in tweet) > max_length) : 
#         max_length = sum(len(i) for i in tweet)
#         max_tweet = tweet

# print('In total ' + str(t) + ' Tweets collected')
# print('On average Tweet is ' + str(round(average_length / t)) + ' charachters long', '\n')
# print('Longest Tweet is:', max_tweet, '\n')
# print('It is', sum(len(i) for i in max_tweet), 'characters long')

Cleaning tweets

In [14]:
# def tweet_cleaner(text):
#     soup =  bs4.BeautifulSoup(text, 'lxml')
#     souped = soup.get_text()
#     try:
#         bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
#     except:
#         bom_removed = souped
        
#     return bom_removed

# data = text_list
# # data = text_list[-int(1e5):]

# # Remove Emails and links to users
# data = [re.sub('\S*@\S*\s?', '', str(sent)) for sent in data]

# # Remove new line characters
# data = [re.sub('\s+', ' ', str(sent)) for sent in data]

# # Remove distracting single quotes
# data = [re.sub("\'", "", str(sent)) for sent in data]

# # Remove https and links
# pat1 = r'@[A-Za-z0-9]+'
# pat2 = r'https?://[A-Za-z0-9./]+'
# combined_pat = r'|'.join((pat1, pat2))

# data = [re.sub(combined_pat, "", str(sent)) for sent in data]

# data = [re.sub(r"https\S+", "", str(sent)) for sent in data]

# data = [re.sub(r"www.[^ ]+", "", str(sent)) for sent in data]


# data = [tweet_cleaner(sent) for sent in data]

In [15]:
# def sent_to_words(sentences) :
#     '''Split sentences into words'''
#     for sentence in sentences :
#         # The main function here is Gensim's simple_preprocess
#         yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# data_words = list(sent_to_words(data))

# # Let's explore what do we have now
# pd.DataFrame(data_words[:5])

In [16]:
# # NLTK stop words collection
# stop_words = stopwords.words('dutch')

# # TODO: Extend stop words collection using native speaker knowledge
# # stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
# stop_words.extend(['rt', 'htt', 'via'])

In [17]:
# print('The number of Dutch stop words avaialable:', len(stop_words), '\n')
# print(stop_words)

In [18]:
# # Build the bigram and trigram models

# # One of the most important parameters here is min_count 
# # If set up, for example, as 5, then only those bigrams and trigrams will be stored that appear 5 times or more

# # Threshold is 
# # TODO: set up various parameters for min_count and threshold
# start = time.time()
# bigram = Phrases(data_words, min_count=5, threshold=10) 
# trigram = Phrases(bigram[data_words], threshold=10)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = Phraser(bigram)
# trigram_mod = Phraser(trigram)
# end = time.time()
# print('Algorithm time =', round((end-start)/60), 'min')

# # See trigram example
# print(trigram_mod[bigram_mod[data_words[5]]])

In [19]:
# # Define functions for stopwords, bigrams, trigrams and lemmatization
# def remove_stopwords(texts) :
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# def make_bigrams(texts) :
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts) :
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']) :
#     texts_out = []
#     for sent in texts :
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [20]:
# # Remove stop words
# start = time.time()
# data_words_nostops = remove_stopwords(data_words)

# # Form bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# nlp = spacy.load('nl', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# end = time.time()

In [21]:
# # Save results
# with open('elections_data_lemmatized', 'wb') as fp:
#     pickle.dump(data_lemmatized, fp)

#### Load the cleaned preprocessed data 

In [22]:
# Load results
with open('elections_data_lemmatized', 'rb') as fp:
    data_lemmatized = pickle.load(fp)

### Get topic from LDA model (Looks into mikhael work for this part)

In [23]:
lda_model = LdaModel.load('lda_model_2d')
all_topic = []
for i in range(lda_model.num_topics):
    all_topic += [lda_model.show_topic(i)]

#### Separate score and words inside each topic for further processing part

In [24]:
def score_num_sep (all_topic = all_topic):
    
    all_topic_words = []
    all_topic_num = []

    for i in range(len(all_topic)):
        one_topic_words = []
        one_topic_num = []
        for j in range(len(all_topic[i])):
            one_topic_words += [all_topic[i][j][0]]
            one_topic_num += [all_topic[i][j][1]]

        all_topic_words += [one_topic_words]
        all_topic_num += [one_topic_num]
    
    return all_topic_words,all_topic_num
    

In [25]:
all_topic_words,all_topic_num = score_num_sep (all_topic = all_topic)

### Group each words inside each topics 
    
    - The input is "Topic Modelling" topics that consists of list of words with the score (LDA, words and score)
    - Moving standard deviation was used for checking whether which words belong to which group inside the topic
    - The output is grouped list of words only, scores only and words and scores.


In [26]:
def topic_grouping (all_topic = all_topic,all_topic_words=all_topic_words,all_topic_num=all_topic_num):

    all_group_topic_all = []
    all_group_topic_words = []
    all_group_topic_num = []

    for topic in range (len(all_topic)):
        all_group_all = []
        all_group_words = []
        all_group_num = []
        count = 0

        while count < len(all_topic[topic]):

            single_group_all = all_topic[topic][count]
            single_group_words_only = (all_topic_words[topic][count],)
            single_group_num_only = (all_topic_num[topic][count],)
            
            # using moving standard deviatiaon for checking group
            range_a = abs(all_topic_num[topic][count] - np.array(all_topic_num[topic][count:len(all_topic_num[topic])]).std())

            for j in range(count,len(all_topic[topic])-1):  

                if count == (len(all_topic[topic])-1):
                    if all_topic_num[topic][-1] <= range_a:
                        single_group_all = (single_group_all) + (all_topic[topic][-1])
                        single_group_words_only = tuple(list(single_group_words_only) + ([all_topic_words[topic][-1]]))
                        single_group_num_only = tuple(list(single_group_num_only) + ([all_topic_num[topic][-1]])) 

                elif count < (len(all_topic[topic])-1):       
                    if all_topic_num[topic][count+1] >= range_a:
                        single_group_all = (single_group_all) + (all_topic[topic][count+1])
                        single_group_words_only = tuple(list(single_group_words_only) + ([all_topic_words[topic][count+1]]))
                        single_group_num_only = tuple(list(single_group_num_only) + ([all_topic_num[topic][count+1]])) 
                        count += 1    

            all_group_all += [single_group_all]
            all_group_words += [single_group_words_only]
            all_group_num += [single_group_num_only]
            count += 1

        all_group_topic_all += [all_group_all]
        all_group_topic_words += [all_group_words]
        all_group_topic_num += [all_group_num]
        
    return all_group_topic_all, all_group_topic_words, all_group_topic_num

In [27]:
all_group_topic_all, all_group_topic_words, all_group_topic_num = topic_grouping (all_topic=all_topic,
                                                                                  all_topic_words=all_topic_words,
                                                                                  all_topic_num=all_topic_num)

#### Sankey diagram layout settings

In [28]:
layout = Layout(width="1000", height="200")
def sankey(margin_top=10, **value):
    """Show SankeyWidget with default values for size and margins"""
    return SankeyWidget(layout=layout,
                        margins=dict(top=margin_top, bottom=0, left=100, right=100),
                        **value)

#### Preparing the dataset list of dicts for sankey diagram using grouped words in the topics

In [29]:
def sankey_links_dict(all_group_topic_words=all_group_topic_words):
    
    links_all = []
    # Colour list dictionary, please add more list (copy paste list or add rows of list) 
    # and colour (add collectiion of list) if needed to produce more colour or if error (out of range) occur
    colour = [
              ['green','lime', 'olive','gold', 'orange','coral', 'red','crimson', 'pink','deeppink','fuchsia', 'lightskyblue',
              'blueviolet','cornflowerblue','blue','navy'], 
              ['coral', 'red','crimson', 'pink','deeppink','green','lime', 'olive','gold', 'orange','fuchsia', 'lightskyblue',
              'blueviolet','cornflowerblue','blue','navy'],
              ['fuchsia','lightskyblue','blueviolet','cornflowerblue','blue','navy','green','lime', 'olive','gold', 'orange',
               'coral', 'red','crimson', 'pink','deeppink'], 
              ['crimson', 'pink','cornflowerblue','blue','navy','green','lime', 'olive','gold', 'orange','coral', 'red',
               'deeppink','fuchsia', 'lightskyblue','blueviolet'],
              ['gold', 'orange','coral''green','lime''blueviolet','cornflowerblue', 'olive', 'red','crimson', 'pink','deeppink',
               'fuchsia', 'lightskyblue','blue','navy'],
              ['crimson', 'pink','cornflowerblue','blue','navy','green','lime', 'olive','gold', 'orange','coral', 'red',
               'deeppink','fuchsia', 'lightskyblue','blueviolet'], 
              ['coral', 'red','crimson', 'pink','deeppink','green','lime', 'olive','gold', 'orange','fuchsia', 'lightskyblue',
              'blueviolet','cornflowerblue','blue','navy'],
              
             ]
    
    # cont the number of all topic
    topic_num_count = len(all_group_topic_words)
    
    # creating links between words by checking the number of words inside the previous and next group of words and link them
    for a in range(topic_num_count):
        groups_count = len(all_group_topic_words[a])
        links = []
        for b in range(groups_count-1):
            
            # check and record number of group of words in current and next
            words_count_1st = len(all_group_topic_words[a][b])
            words_count_2nd = len(all_group_topic_words[a][b+1])

            for c in range(words_count_1st):
                for d in range(words_count_2nd):
                    
                    # creating the dict for sankey diagram visualization
                    dict_1st = {'source': all_group_topic_words[a][b][c], 
                                'target': all_group_topic_words[a][b+1][d], 
                                'value': max(6/words_count_2nd,6/words_count_1st), # use the maximum links values with maximum 6 
                                'color': colour[c][b]} # the use of colour list dictionary 
                    
                    links = links + [dict_1st]

        links_all = links_all + [links]
        
    return links_all

In [30]:
# creating sankey links for each words in LDA topics

links_all = sankey_links_dict(all_group_topic_words=all_group_topic_words)

#### Topic 1 words mapping

In [31]:
sankey(links=links_all[0]).auto_save_png('1.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'ollongren', 'target': 'gewoon', 'va…

#### Topic 2 words mapping

In [32]:
sankey(links=links_all[1]).auto_save_png('2.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'democratie', 'target': 'forum', 'va…

#### Topic 3 words mapping

In [33]:
sankey(links=links_all[2]).auto_save_png('3.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'partij', 'value': …

#### Topic 4 words mapping

In [34]:
sankey(links=links_all[3]).auto_save_png('4.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'pvda', 'value': 6.…

#### Topic 5 words mapping

In [35]:
sankey(links=links_all[4]).auto_save_png('5.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'pvv', 'target': 'stem', 'value': 6.…

#### Topic 6 words mapping

In [36]:
sankey(links=links_all[5]).auto_save_png('6.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'zijlstra', 'value'…

#### Topic 7 words mapping

In [37]:
sankey(links=links_all[6]).auto_save_png('7.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'vvd', 'target': 'wel', 'value': 3.0…

#### Topic 8 words mapping

In [38]:
sankey(links=links_all[7]).auto_save_png('8.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'pvda', 'target': 'rotterdam', 'valu…

#### Topic 9 words mapping

In [39]:
sankey(links=links_all[8]).auto_save_png('9.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'cda', 'target': 'vvd', 'value': 6.0…

#### Topic 10 words mapping

In [40]:
sankey(links=links_all[9]).auto_save_png('10.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'nederland', 'target': 'christenunie…

## Training documents (all tweets) for finding alternative topic (Doc2Vec)

In [41]:
## Document training with doc2vec

## Tag each tweets 
# documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(data_lemmatized)]

## Train documents(tweets) with doc2vec library
# model = Doc2Vec(documents, vector_size=300, window_size = 15, min_count=1, epochs=150, 
#                 sampling_threshold = 1e-5,negative_size = 5, 
#                 workers=32,train_epoch = 150, dm = 0) #0 = dbow; 1 = dmpv 

## Save the training set model for further use
# model.save("d2v.model")

In [42]:
# load the training set model
model = Doc2Vec.load("d2v.model")

In [43]:
# use topics that were found in "Topic Analysis" (LDA) to find similiar topic that match with traininng set model in doc2vec
def similar_topic_doc2vec(word_num=15, all_topic_words = all_topic_words,model=model):
    similar_topic = []

    for topic in range(len(all_topic_words)):
        
        positive_words = list(all_topic_words[topic])
        
        # find words that is not really match with the rest of the words in the topic 
        # and set that as negative words of the topic
        negative_words = model.wv.doesnt_match(all_topic_words[topic])
        
        # set positive words as all of the words in the topic except the negative words
        positive_words.remove(negative_words)
        
        # set the numbers of words for alternative topic that will be generated
        branch_num = word_num
        
        # function to find the most coresponding topics according to the positive and negative words input
        similar = model.wv.most_similar_cosmul(positive =  positive_words,
                                     negative = [negative_words],topn=branch_num)

        similar_topic = similar_topic + [similar]
        
    return similar_topic

In [44]:
# Apply find alternatives topic (we can change the words number!)
similar_topic = similar_topic_doc2vec(word_num=12, all_topic_words = all_topic_words,model=model)

In [45]:
# Separate words and number inside the topics for the alternative topics
all_similar_topic_words,all_similar_topic_num = score_num_sep (all_topic = similar_topic)

In [46]:
# words grouping (clustering) for alternative topics
all_similar_group_topic_all, all_similar_group_topic_words, all_similar_group_topic_num = topic_grouping (all_topic=similar_topic,
                                                                                                          all_topic_words=all_similar_topic_words,
                                                                                                          all_topic_num=all_similar_topic_num)

In [47]:
# creating sankey links for each words in alternative topics
similar_links_all = sankey_links_dict(all_group_topic_words=all_similar_group_topic_words)

#### Alternative Topic for LDA Topic 1

In [48]:
# alternative topic for LDA topic 1
sankey(links=similar_links_all[0]).auto_save_png('alt_1.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'ollongren_ngesteund', 'target': 'ma…

#### Alternative Topic for LDA Topic 2

In [49]:
# alternative topic for LDA topic 2
sankey(links=similar_links_all[1]).auto_save_png('alt_2.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'entameren', 'target': 'blauwe_ogen'…

#### Alternative Topic for LDA Topic 3

In [50]:
# alternative topic for LDA topic 3
sankey(links=similar_links_all[2]).auto_save_png('alt_3.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'leukst', 'target': 'pvd', 'value': …

#### Alternative Topic for LDA Topic 4

In [51]:
# alternative topic for LDA topic 4
sankey(links=similar_links_all[3]).auto_save_png('alt_4.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'derhalve', 'target': 'golddigger', …

#### Alternative Topic for LDA Topic 5

In [52]:
# alternative topic for LDA topic 5
sankey(links=similar_links_all[4]).auto_save_png('alt_5.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'muissti', 'target': 'nbilan', 'valu…

#### Alternative Topic for LDA Topic 6

In [53]:
# alternative topic for LDA topic 6
sankey(links=similar_links_all[5]).auto_save_png('alt_6.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'kanes', 'target': 'derde_termijn', …

#### Alternative Topic for LDA Topic 7

In [54]:
# alternative topic for LDA topic 7
sankey(links=similar_links_all[6]).auto_save_png('alt_7.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'kaaimaneilanden', 'target': 'weeeee…

#### Alternative Topic for LDA Topic 8

In [55]:
# alternative topic for LDA topic 8
sankey(links=similar_links_all[7]).auto_save_png('alt_8.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'nhuishoudelij', 'target': 'wilders_…

#### Alternative Topic for LDA Topic 9

In [56]:
# alternative topic for LDA topic 9
sankey(links=similar_links_all[8]).auto_save_png('alt_9.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'beschaamd', 'target': 'nlach', 'val…

#### Alternative Topic for LDA Topic 10

In [57]:
# alternative topic for LDA topic 10
sankey(links=similar_links_all[9]).auto_save_png('alt_10.png')

SankeyWidget(layout=Layout(height='200', width='1000'), links=[{'source': 'minst_belangrijke', 'target': 'maar…